# Reacton 组件

`PnReacton` 组件可以在 Panel 应用程序中渲染 [Reacton](https://reacton.solara.dev/en/latest/) 组件，无论是在笔记本中还是在部署的服务器上。Reacton 提供了一种以类似 React 的方式编写可重用组件的方法，用于使用 ipywidgets 生态系统（ipywidgets、ipyvolume、bqplot、threejs、leaflet、ipyvuetify 等）构建基于 Python 的 UI。请注意，Reacton 主要是一种编写应用程序的方法。

在笔记本中，这不是必需的，因为 Panel 只是使用常规的笔记本 ipywidget 渲染器。特别是在 JupyterLab 中，以这种方式导入 ipywidgets 扩展可能会干扰 UI 并使 JupyterLab UI 无法使用，因此请谨慎启用扩展。

底层实现为`panel.pane.Reacton`，参数基本一致，参考文档：https://panel.holoviz.org/reference/panes/Reacton.html


In [ ]:
##ignore
%load_ext vuepy
from panel_vuepy import vpanel


## 基本用法

`panel_vuepy` 函数会自动将任何 Reacton 组件转换为可显示的面板，同时保持其所有交互功能：


In [ ]:
%%vuepy_run --plugins vpanel --show-code
<template>
  <PnReacton :object="ButtonClick()" />
</template>
<script lang='py'>
import reacton
import reacton.ipywidgets as w

@reacton.component
def ButtonClick():
    # 首次渲染时返回0，之后返回set_clicks的最后一个参数
    clicks, set_clicks = reacton.use_state(0)
    
    def my_click_handler():
        # 用新的clicks值触发新的渲染
        set_clicks(clicks+1)

    button = w.Button(description=f"点击了 {clicks} 次",
                    on_click=my_click_handler)
    return button
</script>


## 结合 Reacton 和 Panel 组件

Reacton 可以与 Panel 组件结合使用，但我们需要做两个修改：

1. Panel 组件必须使用 `pn.ipywidget` 包装器包装为 ipywidget（这需要 `jupyter_bokeh`）。
2. 包装后的 Panel 组件必须添加到 reacton 布局组件中。

在下面的示例中，我们将 `reacton.ipywidgets.Button` 替换为 `PnButton`，然后用 `pn.ipywidget` 和 `reacton.ipywidgets.VBox` 包装它：


In [ ]:
%%vuepy_run --plugins vpanel --show-code
<template>
  <PnReacton :object="PanelButtonClick()" :height="50" />
</template>
<script lang='py'>
import reacton
import reacton.ipywidgets as w
import panel as pn
from panel_vuepy import vpanel

@reacton.component
def PanelButtonClick():
    # 首次渲染时返回0，之后返回set_clicks的最后一个参数
    clicks, set_clicks = reacton.use_state(0)
    
    def my_click_handler(event):
        # 用新的clicks值触发新的渲染
        set_clicks(clicks+1)

    button = vpanel.widgets.Button(name=f'点击了 {clicks} 次')
    button.on_click(my_click_handler)

    return w.VBox(children=[pn.ipywidget(button)])
</script>


## 复杂示例

可以在 Reacton 中构建更复杂的应用程序并在 Panel 中显示。以下是 Reacton 文档中的计算器示例。

### 逻辑


In [ ]:
%%vuepy_run --plugins vpanel --show-code
<template>
  <PnReacton :object="Calculator()" :width="500" :height="250" />
</template>
<script lang='py'>
import reacton
import reacton.ipywidgets as w
import ast
import dataclasses
import operator
from typing import Any, Optional

DEBUG = False
operator_map = {
    "x": operator.mul,
    "/": operator.truediv,
    "+": operator.add,
    "-": operator.sub,
}

@dataclasses.dataclass(frozen=True)
class CalculatorState:
    input: str = ""
    output: str = ""
    left: float = 0
    right: Optional[float] = None
    operator: Any = operator.add
    error: str = ""


initial_state = CalculatorState()


def calculate(state: CalculatorState):
    result = state.operator(state.left, state.right)
    return dataclasses.replace(state, left=result)


def calculator_reducer(state: CalculatorState, action):
    action_type, payload = action
    if DEBUG:
        print("reducer", state, action_type, payload)  # noqa
    state = dataclasses.replace(state, error="")

    if action_type == "digit":
        digit = payload
        input = state.input + digit
        return dataclasses.replace(state, input=input, output=input)
    elif action_type == "percent":
        if state.input:
            try:
                value = ast.literal_eval(state.input)
            except Exception as e:
                return dataclasses.replace(state, error=str(e))
            state = dataclasses.replace(state, right=value / 100)
            state = calculate(state)
            output = f"{value / 100:,}"
            return dataclasses.replace(state, output=output, input="")
        else:
            output = f"{state.left / 100:,}"
            return dataclasses.replace(state, left=state.left / 100, output=output)
    elif action_type == "negate":
        if state.input:
            input = state.output
            input = input[1:] if input[0] == "-" else "-" + input
            output = input
            return dataclasses.replace(state, input=input, output=output)
        else:
            output = f"{-state.left:,}"
            return dataclasses.replace(state, left=-state.left, output=output)
    elif action_type == "clear":
        return dataclasses.replace(state, input="", output="")
    elif action_type == "reset":
        return initial_state
    elif action_type == "calculate":
        if state.input:
            try:
                value = ast.literal_eval(state.input)
            except Exception as e:
                return dataclasses.replace(state, error=str(e))
            state = dataclasses.replace(state, right=value)
        state = calculate(state)
        output = f"{state.left:,}"
        state = dataclasses.replace(state, output=output, input="")
        return state
    elif action_type == "operator":
        if state.input:
            state = calculator_reducer(state, ("calculate", None))
            state = dataclasses.replace(state, operator=payload, input="")
        else:
            # 例如 2+3=*= 应该给出 5,25
            state = dataclasses.replace(state, operator=payload, right=state.left)
        return state
    else:
        print("无效操作", action)  # noqa
        return state

@reacton.component
def Calculator():
    state, dispatch = reacton.use_reducer(calculator_reducer, initial_state)
    with w.VBox() as main:
        w.HTML(value="<b>使用 Reacton 的计算器</b>")
        with w.VBox():
            w.HTML(value=state.error or state.output or "0")
            with w.HBox():
                if state.input:
                    w.Button(description="C", on_click=lambda: dispatch(("clear", None)))
                else:
                    w.Button(description="AC", on_click=lambda: dispatch(("reset", None)))
                w.Button(description="+/-", on_click=lambda: dispatch(("negate", None)))
                w.Button(description="%", on_click=lambda: dispatch(("percent", None)))
                w.Button(description="/", on_click=lambda: dispatch(("operator", operator_map["/"])))

            column_op = ["x", "-", "+"]
            for i in range(3):
                with w.HBox():
                    for j in range(3):
                        digit = str(j + (2 - i) * 3 + 1)
                        w.Button(description=digit, on_click=lambda digit=digit: dispatch(("digit", digit)))
                    op_symbol = column_op[i]
                    op = operator_map[op_symbol]
                    w.Button(description=op_symbol, on_click=lambda op=op: dispatch(("operator", op)))
            with w.HBox():
                w.Button(description="0", on_click=lambda: dispatch(("digit", "0")))
                w.Button(description=".", on_click=lambda: dispatch(("digit", ".")))
                w.Button(description="=", on_click=lambda: dispatch(("calculate", None)))

    return main
</script>


## 使用 ipyvuetify

Reacton 也可以与 ipyvuetify 结合使用，创建更美观的界面：


In [ ]:
%%vuepy_run --plugins vpanel --show-code
<template>
  <PnReacton :object="CalculatorVuetify()" :width="500" :height="300" />
</template>
<script lang='py'>
import reacton
import reacton.ipywidgets as w
import reacton.ipyvuetify as v
import ast
import dataclasses
import operator
from typing import Any, Optional

DEBUG = False
operator_map = {
    "x": operator.mul,
    "/": operator.truediv,
    "+": operator.add,
    "-": operator.sub,
}

@dataclasses.dataclass(frozen=True)
class CalculatorState:
    input: str = ""
    output: str = ""
    left: float = 0
    right: Optional[float] = None
    operator: Any = operator.add
    error: str = ""


initial_state = CalculatorState()


def calculate(state: CalculatorState):
    result = state.operator(state.left, state.right)
    return dataclasses.replace(state, left=result)


def calculator_reducer(state: CalculatorState, action):
    action_type, payload = action
    if DEBUG:
        print("reducer", state, action_type, payload)  # noqa
    state = dataclasses.replace(state, error="")

    if action_type == "digit":
        digit = payload
        input = state.input + digit
        return dataclasses.replace(state, input=input, output=input)
    elif action_type == "percent":
        if state.input:
            try:
                value = ast.literal_eval(state.input)
            except Exception as e:
                return dataclasses.replace(state, error=str(e))
            state = dataclasses.replace(state, right=value / 100)
            state = calculate(state)
            output = f"{value / 100:,}"
            return dataclasses.replace(state, output=output, input="")
        else:
            output = f"{state.left / 100:,}"
            return dataclasses.replace(state, left=state.left / 100, output=output)
    elif action_type == "negate":
        if state.input:
            input = state.output
            input = input[1:] if input[0] == "-" else "-" + input
            output = input
            return dataclasses.replace(state, input=input, output=output)
        else:
            output = f"{-state.left:,}"
            return dataclasses.replace(state, left=-state.left, output=output)
    elif action_type == "clear":
        return dataclasses.replace(state, input="", output="")
    elif action_type == "reset":
        return initial_state
    elif action_type == "calculate":
        if state.input:
            try:
                value = ast.literal_eval(state.input)
            except Exception as e:
                return dataclasses.replace(state, error=str(e))
            state = dataclasses.replace(state, right=value)
        state = calculate(state)
        output = f"{state.left:,}"
        state = dataclasses.replace(state, output=output, input="")
        return state
    elif action_type == "operator":
        if state.input:
            state = calculator_reducer(state, ("calculate", None))
            state = dataclasses.replace(state, operator=payload, input="")
        else:
            # 例如 2+3=*= 应该给出 5,25
            state = dataclasses.replace(state, operator=payload, right=state.left)
        return state
    else:
        print("无效操作", action)  # noqa
        return state

@reacton.component
def CalculatorVuetify():
    state, dispatch = reacton.use_reducer(calculator_reducer, initial_state)
    with v.Card(elevation=10, class_="ma-4") as main:
        with v.CardTitle(children=["计算器"]):
            pass
        with v.CardSubtitle(children=["使用 ipyvuetify 和 Reacton"]):
            pass
        with v.CardText():
            with v.Row():
                with v.Col(cols=12):
                    v.TextField(
                        value=state.error or state.output or "0",
                        readonly=True,
                        outlined=True,
                        class_="text-right",
                    )
            
            btn_class = "ma-1"
            btn_color = "primary"
            with v.Row():
                with v.Col(cols=3):
                    if state.input:
                        btn = v.Btn(children=["C"], color=btn_color, class_=btn_class)
                        v.use_event(btn, 'click', lambda _, __, ___: dispatch(("clear", None)))
                    else:
                        btn = v.Btn(children=["AC"], color=btn_color, class_=btn_class)
                        v.use_event(btn, 'click', lambda _, __, ___: dispatch(("reset", None)))
                with v.Col(cols=3):
                    btn = v.Btn(children=["+/-"], color=btn_color, class_=btn_class)
                    v.use_event(btn, 'click', lambda _, __, ___: dispatch(("negate", None)))
                with v.Col(cols=3):
                    btn = v.Btn(children=["%"], color=btn_color, class_=btn_class)
                    v.use_event(btn, 'click', lambda _, __, ___: dispatch(("percent", None)))
                with v.Col(cols=3):
                    btn = v.Btn(children=["/"], color="error", class_=btn_class)
                    v.use_event(btn, 'click', lambda _, __, ___: dispatch(("operator", operator_map["/"])))
            
            column_op = ["x", "-", "+"]
            for i in range(3):
                with v.Row():
                    for j in range(3):
                        digit = str(j + (2 - i) * 3 + 1)
                        with v.Col(cols=3):
                            btn = v.Btn(children=[digit], color="secondary", class_=btn_class)
                            v.use_event(btn, 'click', lambda _, __, ___, digit=digit: dispatch(("digit", digit)))
                    with v.Col(cols=3):
                        op_symbol = column_op[i]
                        op = operator_map[op_symbol]
                        btn = v.Btn(children=[op_symbol], color="error", class_=btn_class)
                        v.use_event(btn, 'click', lambda _, __, ___, op=op: dispatch(("operator", op)))
            
            with v.Row():
                with v.Col(cols=3):
                    pass
                with v.Col(cols=3):
                    btn = v.Btn(children=["0"], color="secondary", class_=btn_class)
                    v.use_event(btn, 'click', lambda _, __, ___: dispatch(("digit", "0")))
                with v.Col(cols=3):
                    btn = v.Btn(children=["."], color="secondary", class_=btn_class)
                    v.use_event(btn, 'click', lambda _, __, ___: dispatch(("digit", ".")))
                with v.Col(cols=3):
                    btn = v.Btn(children=["="], color="success", class_=btn_class)
                    v.use_event(btn, 'click', lambda _, __, ___: dispatch(("calculate", None)))
    
    return main
</script>


## API

### 属性

| 属性名           | 说明                          | 类型                                                           | 默认值 |
| --------------- | ----------------------------- | ---------------------------------------------------------------| ------- |
| object          | 被显示的 Reacton 组件对象      | ^[object]                                                      | None |
| default_layout  | 包装图表和小部件的布局        | ^[pn.layout.Panel]                                             | Row |
| sizing_mode     | 尺寸调整模式                  | ^[str]                                                         | 'fixed'  |
| width           | 宽度                          | ^[int, str]                                                    | None    |
| height          | 高度                          | ^[int, str]                                                    | None    |
| min_width       | 最小宽度                      | ^[int]                                                         | None    |
| min_height      | 最小高度                      | ^[int]                                                         | None    |
| max_width       | 最大宽度                      | ^[int]                                                         | None    |
| max_height      | 最大高度                      | ^[int]                                                         | None    |
| margin          | 外边距                        | ^[int, tuple]                                                  | 5       |
| css_classes     | CSS类名列表                   | ^[list]                                                        | []      |

### Events

| 事件名 | 说明                  | 类型                                   |
| ---   | ---                  | ---                                    |

### Slots

| 插槽名   | 说明               |
| ---     | ---               |
| default | 自定义默认内容      |

### 方法

| 属性名 | 说明 | 类型 |
| --- | --- | --- |


In [ ]:
##ignore
import numpy as np
import panel as pn
import reacton
import reacton.ipywidgets as w

pn.extension('ipywidgets')

@reacton.component
def ButtonClick():
    # 首次渲染时返回0，之后返回set_clicks的最后一个参数
    clicks, set_clicks = reacton.use_state(0)
    
    def my_click_handler():
        # 用新的clicks值触发新的渲染
        set_clicks(clicks+1)

    button = w.Button(description=f"点击了 {clicks} 次",
                    on_click=my_click_handler)
    return button

pn.panel(ButtonClick())